In [1]:
import SimpleITK as sitk
import numpy as np
import cv2
from motionSegmentation import simpleSolvery
import medImgProc.processFunc as pf
import os
import shutil as sh

SyntaxError: can't use starred expression here (BsplineFourier.py, line 2404)

In [ ]:
start_slice = 26
end_slice   = 27
anchor_1    = 9
anchor_2    = 20
start_t     = 0
end_t       = 19
label_t     = anchor_1
case_path   = "/Users/nureizzatyhamzaid/Desktop/FYP_23/Disease/Case 95/Pre_Slices" 
paras_path  = case_path + '/paras/'
label_file  = 'label{}.png'.format(label_t)

In [ ]:

"""
# rm_arrowsh
for i in range(start_slice, end_slice + 1):
    path  = paras_path + 'para{}/'.format(i)
    path2 = paras_path + 'para{}_new/'.format(i)
    if not os.path.exists(path2):
        os.makedirs(path2)

    dirs = os.listdir(path)
    for f in dirs:
        img = cv2.imread(paras_path + 'para{}/'.format(i) + f, 0)
        if img is not None and f[0] != 'S':
            
            listCoordmax0 = np.where(img > 200)
            listCoordmax = list(zip(listCoordmax0[0], listCoordmax0[1]))
            listfilling = list(zip(listCoordmax0[0], listCoordmax0[1] - 5))
            
            for pos1, pos2 in zip(listCoordmax, listfilling):
                img[pos1] = img[pos2]
            # print(path+f)
            
            cv2.imwrite(path2 + f, img)
"""

In [ ]:
# rm_arrows(02Mar)
"""
for i in range(start_slice, end_slice + 1):
    path  = paras_path + 'para{}/'.format(i)
    path2 = paras_path + 'para{}_new/'.format(i)
    if not os.path.exists(path2):
        os.makedirs(path2)

    dirs = os.listdir(path)
    for f in dirs:
        img = cv2.imread(paras_path + 'para{}/'.format(i) + f, 0)
        if img is not None and f[0] != 'S':
            
            img = img[40:390, 40:476-30, ...]
            # print(path+f)
            cv2.imwrite(path2 + f, img)
"""

In [ ]:
# convert.py
for i in range(start_slice, end_slice + 1): 
    label_1 = 255 - cv2.imread(case_path + 'time{0:03d}/Segmented/Segmented Slice{1:03d}.png'.format(anchor_1, i))
    # print('time{0:03d}/segmented/Segmented Slice{1:03d}.png'.format(anchor_1, i).format(anchor_1, i))
    label_2 = 255 - cv2.imread(case_path + 'time{0:03d}/Segmented/Segmented Slice{1:03d}.png'.format(anchor_2, i))
    # print(label_1.shape)
    label_1 = np.uint8(label_1 == 0)*100 + 50
    label_2 = np.uint8(label_2 == 0)*100 + 50

    cv2.imwrite(paras_path + 'para{}_new/label{}.png'.format(i, anchor_1), label_1)
    cv2.imwrite(paras_path + 'para{}_new/label{}.png'.format(i, anchor_2), label_2)

In [ ]:
# convert.py(02Mar)
""" 
for i in range(start_slice, end_slice + 1): 
    label_1 = 255 - cv2.imread(case_path + 'time{0:03d}/Segmented/Segmented Slice{1:03d}.png'.format(anchor_1, i))
    # print('time{0:03d}/segmented/Segmented Slice{1:03d}.png'.format(anchor_1, i).format(anchor_1, i))
    label_2 = 255 - cv2.imread(case_path + 'time{0:03d}/Segmented/Segmented Slice{1:03d}.png'.format(anchor_2, i))
    # print(label_1.shape)
    label_1 = label_1[40:390, 40:476-30, ...]
    label_2 = label_2[40:390, 40:476-30, ...]
    label_1 = np.uint8(label_1 == 0)*100 + 50
    label_2 = np.uint8(label_2 == 0)*100 + 50
    cv2.imwrite(paras_path + 'para{}_new/label{}.png'.format(i, anchor_1), label_1)
    cv2.imwrite(paras_path + 'para{}_new/label{}.png'.format(i, anchor_2), label_2)
"""


In [ ]:
# solve.py
for slice in range(start_slice, end_slice + 1):
    anchor= [[anchor_1 - 1 , anchor_2 - 1,paras_path + 'para{}_new/label{}.png'.format(slice, anchor_1), paras_path + 'para{}_new/label{}.png'.format(slice, anchor_2)]]
    savePath = paras_path + 'para{}_new/'.format(slice)
    if not os.path.exists(savePath + 'scale.txt'):
        sh.copyfile(paras_path + '../scale.txt', savePath + 'scale.txt')
    simpleSolver(savePath,startstep=1,endstep=6,fileScale=None,
                 getCompoundTimeList=range(start_t, end_t + 1),
                 compoundSchemeList=None,fftLagrangian=True,
                 pngFileFormat=None,period=None,maskImg=True,
                 anchor=anchor,bgrid=30.,imgfmt='uint8',
                 finalShape=None,fourierTerms=4,twoD=True)
    # change everything to appropriate dimensions before mp.image.save

In [ ]:
# transform.py
# Transforms the labels to the labels of the appropriate time points

for slice in range(start_slice, end_slice + 1):
    label = cv2.imread(paras_path + '/para{}_new/'.format(slice) + label_file,  0)
    stlFile = cv2.GaussianBlur(label, (5,5), 20)
    # print(label)
    for t in range(start_t, end_t + 1):
        if t != label_t-1:
            trfFile = paras_path + '/para{}_new/bsfTransform/t{}to{}.txt'.format(slice, t, label_t - 1)
            # print(trfFile)
            pf.transform_img2img(stlFile, trfFile,savePath=paras_path + 'para{}_new'.format(slice),fileName='trf{}'.format(t), scale=0.303030)    # scale should be the same as that in 'scale.txt'
            trf_dir = paras_path + '/para{}_new/trfs/'.format(slice)
            if not os.path.exists(trf_dir):
                os.makedirs(trf_dir)
            img = sitk.ReadImage(paras_path + '/para{}_new/trf{}.mha'.format(slice, t))
            array = sitk.GetArrayFromImage(img)
            array = np.uint8(array > 80)*255
            cv2.imwrite(trf_dir + 'trf{}.png'.format(t + 1), array)
        else:
            print(" ")
            trf_dir = paras_path + '/para{}_new/trfs/'.format(slice)
            array = np.uint8(label > 80)*255
            cv2.imwrite(trf_dir + 'trf{}.png'.format(t + 1), array)

In [ ]:
# masking.py
for slice in range(start_slice, end_slice + 1):
    # print(label)
    
    for t in range(start_t + 1, end_t + 1):
        # pic_file  = case_path + 'time{0:03d}/'.format(t) + 'slice{0:03d}time{1:03d}.png'.format(slice, t)
        label = cv2.imread(paras_path + 'para{}_new/'.format(slice) + 'trfs/trf{}.png'.format(t),  0)
        pic_file = paras_path + 'para{}_new/'.format(slice) + 'time{0:03d}.png'.format(t)
        # print(pic_file)
        pic = cv2.imread(pic_file)
        pic[..., 2] = 0.8*pic[..., 2] + 0.2 * label             # overlays the  labels to the original picture in red
        # dst = pic[..., 2] + 50 * np.uint8(label == label.max())
        masked_path = paras_path + 'para{}_new/'.format(slice) + 'masked/'
        if not os.path.exists(masked_path):
            os.makedirs(masked_path)
        # print(masked_path)
        cv2.imwrite(masked_path + 'slice{0:03d}time{1:03d}.png'.format(slice, t), pic)